In [1]:
!pip install concrete-ml==1.9.0 pandas scikit-learn xgboost joblib networkx

#### 👤 2. User – Xử lý & mã hóa dữ liệu
- Load Client
- Load dữ liệu
- Mã hóa đặc trưng bằng `public key (pk)`.
- Lưu lại để chuyển cho CPS

#####2.1 Load Client

In [2]:
import zipfile
import os
from concrete import fhe
from concrete.fhe import Value

path="/content/drive/MyDrive/Colab Notebooks/XGBoost-FHE-ConcreteML-Phát hiện gian lận thẻ tín dụng /Data/Analyst/fhe_artifacts/client.zip"

client = fhe.Client.load(path)
print("✅ Client FHE đã load thành công!")

✅ Client FHE đã load thành công!


##### 2.2 Load dữ liệu
- Mã hóa đặc trưng bằng `public key (pk)`.
- Lưu lại để chuyển cho CPS

In [3]:
import pandas as pd
import numpy as np

columns = ["Time"] + [f"V{i}" for i in range(1,29)] + ["Amount", "Class"]
data = []

# Tạo 5 bản mẫu giả lập
np.random.seed(42)
for i in range(5):
    row = [
        np.random.randint(0,2),                   # Time: 0 hoặc 1
        *np.round(np.random.normal(0,1,28), 6),   # V1-V28: giá trị float ~ N(0,1)
        round(np.random.uniform(0,500),2),        # Amount: float từ 0-500
        np.random.randint(0,2)                     # Class: 0 hoặc 1
    ]
    data.append(row)

df_fake = pd.DataFrame(data, columns=columns)
print(df_fake)

   Time        V1        V2        V3        V4        V5        V6        V7  \
0     0 -0.550234  0.515433  0.473861  1.368450 -0.916827 -0.124147 -2.010963   
1     0 -0.386473 -1.158770  0.566113 -0.704453 -1.377939 -0.353117 -0.461466   
2     0  0.930584  0.677767  0.698440  0.173602  0.662285  0.241122 -0.168205   
3     1  0.135754  0.608090  0.704981  0.360923 -1.466968  0.892629 -0.105257   
4     1  0.817874 -1.054604 -0.758327  0.457416 -0.064378  0.344902 -0.080087   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0 -0.492803  0.392580  ...  1.134339 -0.104746 -0.525123  1.912771 -2.026720   
1  0.066657 -0.176286  ... -1.509957  0.721502 -0.843420 -0.513325 -0.043537   
2  1.164769 -0.245748  ... -1.380717 -0.280806 -0.059817  0.961174  1.794281   
3 -0.955346 -0.414765  ...  1.945116 -0.153336 -0.906987 -0.817354 -0.483123   
4 -0.241380  1.433515  ...  1.218050 -0.389458 -0.442802  0.378227  1.933840   

        V26       V27       V28 

##### 2.3 Mã hóa đặc trưng bằng `public key (pk)`.

In [6]:
import joblib
X_fake = df_fake.drop(columns=["Class"])
# Chuẩn hóa với scaler đã train
scaler = joblib.load("/content/drive/MyDrive/Colab Notebooks/XGBoost-FHE-ConcreteML-Phát hiện gian lận thẻ tín dụng /Data/Analyst/models/scaler.joblib")
X_fake = scaler.transform(X_fake)

# Chuyển float -> integer theo bitwidth n_bits
n_bits = 8
max_int = 2**n_bits - 1
X_fake = np.round((X_fake - X_fake.min(axis=0)) /
                      (X_fake.max(axis=0) - X_fake.min(axis=0)) * max_int).astype(np.int64)

Y_fake = df_fake["Class"].to_numpy()
X_fake_ = []
for i in range(X_fake.shape[0]):
    x_row = X_fake[i].reshape(1, -1)   # (1, 30)
    X_fake_.append(x_row)
ciphertexts = [client.encrypt(x) for x in X_fake_]



##### 2.4 Lưu lại để chuyển cho CPS

In [5]:
path="/content/drive/MyDrive/Colab Notebooks/XGBoost-FHE-ConcreteML-Phát hiện gian lận thẻ tín dụng /Data/User/data_enc"
os.makedirs(path, exist_ok=True)
for i, ct in enumerate(ciphertexts):
    with open(f"{path}/sample_{i+1}.bin", "wb") as f:
        f.write(ct.serialize())